In [40]:
import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import Image
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Data Preparation

In [41]:
def resize_and_pad_image(file_path, max_dimension, target_size=(100, 100)):
    # Open the image
    image = Image.open(file_path)
    
    # Get original width and height
    original_width, original_height = image.size
    
    # Calculate aspect ratio
    aspect_ratio = original_width / original_height
    
    # Determine new dimensions
    if original_width >= original_height:
        new_width = max_dimension
        new_height = int(max_dimension / aspect_ratio)
    else:
        new_height = max_dimension
        new_width = int(max_dimension * aspect_ratio)
    
    # Resize the image while maintaining aspect ratio
    resized_image = image.resize((new_width, new_height), Image.LANCZOS)
    
    # Create a new blank white image with the target size
    new_image = Image.new("RGB", target_size, color="white")

    # Paste the resized image onto the center of the blank white image
    position = ((target_size[0] - resized_image.size[0]) // 2, (target_size[1] - resized_image.size[1]) // 2)
    new_image.paste(resized_image, position)
    
    return new_image

In [42]:
# Load and preprocess images from file paths
def load_image(file_path, max_dimension=100):
    resized_image = resize_and_pad_image(file_path, max_dimension)
    image_array = img_to_array(resized_image) / 255.0
    return image_array

In [43]:
def prepare_data(file_path):
    # Read the csv file
    df = pd.read_csv(file_path)

    # Trim dataset for faster testing for now
    #df = df.sample(n=50, random_state=42)

    # Append data/ in front of every image file path
    df['image:FILE'] = 'data/' + df['image:FILE']
    
    # Add new column with loaded image
    df['image'] = df['image:FILE'].apply(lambda x: load_image(x))

    # Shuffle the dataframe
    df = df.sample(frac=1.0, random_state=42)

    X = np.stack(df['image'].to_numpy())
    y = df['category'].to_numpy()

    return X, y


In [44]:
X_train, y_train = prepare_data('data/test.csv')
X_val, y_val = prepare_data('data/val.csv')
X_test, y_test = prepare_data('data/test.csv')

# Train Model

In [45]:
# To ensure we have reproducable results for each model execution
import os
os.environ['TF_NUM_INTEROP_THREADS'] = '1'

In [46]:
def CNN_train():
    # Set random seed for TensorFlow operations
    tf.random.set_seed(42)
    
    # Define the CNN architecture
    model = models.Sequential([
        layers.Input(shape=(100, 100, 3)),
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(30)  # 30 unique labels in dataset
    ])

    # Compile the model
    model.compile(optimizer='adam',
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy'])
    
    # Train the model
    model.fit(X_train, y_train, epochs=15, validation_data=(X_val, y_val))

    # Evaluate the model
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
    print(f'Test accuracy: {test_acc}')
    print(f'Test loss: {test_loss}')

    # Save the model
    model.save('CNN_model.keras')

In [47]:
CNN_train()

Epoch 1/15
188/188 ━━━━━━━━━━━━━━━━━━━━ 44s 222ms/step - accuracy: 0.0858 - loss: 3.2567 - val_accuracy: 0.2177 - val_loss: 2.7509
Epoch 2/15
188/188 ━━━━━━━━━━━━━━━━━━━━ 39s 207ms/step - accuracy: 0.3127 - loss: 2.3279 - val_accuracy: 0.3123 - val_loss: 2.3636
Epoch 3/15
188/188 ━━━━━━━━━━━━━━━━━━━━ 39s 207ms/step - accuracy: 0.4471 - loss: 1.8910 - val_accuracy: 0.3827 - val_loss: 2.2214
Epoch 4/15
188/188 ━━━━━━━━━━━━━━━━━━━━ 37s 195ms/step - accuracy: 0.5635 - loss: 1.4902 - val_accuracy: 0.4260 - val_loss: 2.1461
Epoch 5/15
188/188 ━━━━━━━━━━━━━━━━━━━━ 40s 212ms/step - accuracy: 0.6684 - loss: 1.1484 - val_accuracy: 0.4613 - val_loss: 2.3251
Epoch 6/15
188/188 ━━━━━━━━━━━━━━━━━━━━ 36s 193ms/step - accuracy: 0.7584 - loss: 0.8003 - val_accuracy: 0.4650 - val_loss: 2.3867
Epoch 7/15
188/188 ━━━━━━━━━━━━━━━━━━━━ 36s 193ms/step - accuracy: 0.8094 - loss: 0.5990 - val_accuracy: 0.4770 - val_loss: 2.7153
Epoch 8/15
188/188 ━━━━━━━━━━━━━━━━━━━━ 37s 195ms/step - accuracy: 0.8652 - loss: 0